In [1]:
import joblib
import pandas as pd

FEATURES = [
    "duration",
    "credit_amount",
    "age",
    "checking_status",
    "employment",
    "savings_status",
    "purpose",
]

model = joblib.load("../artifacts/models/model_v1.joblib")

sample = {
    "duration": 24,
    "credit_amount": 4500,
    "age": 28,
    "checking_status": "no checking",
    "employment": "unemployed",
    "savings_status": "<100",
    "purpose": "vacation/others"
}

X_new = pd.DataFrame([sample], columns=FEATURES)

proba_good = float(model.predict_proba(X_new)[:, 1][0])
decision = "APROBADO" if proba_good >= 0.5 else "RECHAZADO"

print({"proba_good": round(proba_good, 4), "decision": decision})
# --- SIMULACIÓN DE VARIOS EJEMPLOS (SERVING) ---
examples = [
    # Caso 1: Alto Riesgo (Joven, desempleado, monto alto)
    {"duration": 48, "credit_amount": 15000, "age": 20, "checking_status": "<0", "employment": "unemployed", "savings_status": "<100", "purpose": "business"},
    # Caso 2: Bajo Riesgo (Mayor, empleo estable, monto bajo)
    {"duration": 12, "credit_amount": 2500, "age": 55, "checking_status": ">=200", "employment": ">=7", "savings_status": ">=1000", "purpose": "car"}
]

print("\n=== REPORTE DE PREDICCIONES (Nuevas Instancias) ===")
for i, sample in enumerate(examples):
    X_new = pd.DataFrame([sample], columns=FEATURES)
    proba = model.predict_proba(X_new)[:, 1][0]
    decision = "APROBADO" if proba >= 0.5 else "RECHAZADO"
    print(f"Cliente #{i+1} | Probabilidad de Buen Pago: {proba:.2%} -> Decisión: {decision}")


{'proba_good': 0.7675, 'decision': 'APROBADO'}

=== REPORTE DE PREDICCIONES (Nuevas Instancias) ===
Cliente #1 | Probabilidad de Buen Pago: 42.63% -> Decisión: RECHAZADO
Cliente #2 | Probabilidad de Buen Pago: 87.46% -> Decisión: APROBADO


# Fase 4: Predicción con Nuevas Instancias (Model Serving)

## 4.1 Inferencia en Producción
La fase de "Serving" simula la puesta en producción del modelo. Se define una **función de predicción** que:
1.  Recibe datos crudos de un nuevo cliente.
2.  Carga los artefactos (`Pipeline` + `Modelo`) sin reentrenar.
3.  Aplica transformaciones y genera la probabilidad de riesgo.

## 4.2 Escenarios de Uso (Test de Estrés)
Para validar la robustez, se simulan múltiples perfiles de clientes:
* **Perfil A (Riesgo Alto):** Jóvenes, sin empleo, montos altos.
* **Perfil B (Riesgo Bajo):** Adultos, empleo estable, montos bajos.

Esto demuestra la capacidad del modelo para generalizar y tomar decisiones automáticas (Aprobado/Rechazado).